Before running this noetbook makes sure that the following packages are installed in your Python enviroment:
* `snowflake-snowpark-python`
* `snowflake-ml-python`

You also need to make sure the `demo_data.py`file is in the same directory as this notebook.

In [1]:
# Import python packages

from snowflake.ml.feature_store import (
    FeatureStore,
    CreationMode)

from snowflake.ml.registry import Registry

# from snowflake.snowpark import Session
import snowflake.snowpark.functions as snow_funcs

# Python script stored on a Snowflake stage
from demo_data import generate_demo_data

# Get the Snowpark session
from snowflake.snowpark import Session

In [2]:
db_name = "SNOWPARK_DEMO_DB"
schema_name = "SIMPLE_ML_SCHEMA"
fs_schema_name = "SIMPLE_FS_SCHEMA"
mr_schema_name = "SIMPLE_MR_SCHEMA"
wh_name = "SIMPLE_ML_WH"

CONNECTION_NAME = '<YOUR CONNECTION NAME>' # Name of the connection in connections.toml to be used to connect to Snowflake

session = Session.builder.config("connection_name", CONNECTION_NAME).create()
session.use_schema(f'{db_name}.{schema_name}')
session.use_warehouse(wh_name)

## Generate new customers for inference

Generate 100 new customers to use, we will have to wait around one minute before they are part of our feature store (the schedule we setup during registring the features)

In [ ]:
# Generate new customers for year 2024
generate_demo_data(session, num_customers=100, month=9, start_year=2024, end_year=2024)

Greate a Spine DataFrame with the new customers, which will be used to get the features we have for them

In [ ]:
# Retrieve new customers
new_customers_df = session.table(f'{db_name}.{schema_name}.CUSTOMER_LIFE_TIME_VALUE').filter(snow_funcs.col('YEAR_MONTH')=='20249').select('EMAIL')
new_customers_df.show()

Connect to the Feature Store

In [5]:
# Connect to Feature Store
fs = FeatureStore(
    session=session, 
    database=db_name, 
    name=fs_schema_name, 
    default_warehouse=wh_name,
    creation_mode=CreationMode.FAIL_IF_NOT_EXIST,
)

Get the feature views that has the features we want to use

In [6]:
cust_fv = fs.get_feature_view(name="CUSTOMER_GENERAL_DATA_FEATURES",
                                version="V1")
behavior_fv = fs.get_feature_view(name="CUSTOMER_BEHAVIOR_DATA_FEATURES",
                                version="V1")

Retrieve features using the Spine DataFrame (can take up to a minute until values appear)  
Assumption is that you only have the unique-id EMAIL and need to retrieve the features to score the model

In [ ]:

new_customers_features = fs.retrieve_feature_values(new_customers_df, features=[cust_fv, behavior_fv])
new_customers_features.show()

Connect to the Model Registry

In [ ]:
ml_reg = Registry(session=session, database_name=db_name, schema_name=mr_schema_name)
ml_reg.show_models()

Get a reference to the default version of the model

In [9]:
registered_model = ml_reg.get_model("CUSTOMER_LTV_MODEL").default

Check which functions we can use and the inputs/outputs for them

In [ ]:
registered_model.show_functions()

Create predictions from registered model given the retrieved features, we are saving them as a table in Snowflake

In [ ]:

new_predictions = registered_model.run(new_customers_features, function_name='predict')
new_predictions.write.save_as_table('MY_PREDICTIONS', mode='overwrite')
session.table('MY_PREDICTIONS').show()

In [17]:
session.close()